In [6]:
#install.packages('reticulate')

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [51]:
list.files('..')

[1] "add.py"                                                                                                   
 [2] "baseline_template.bes"                                                                                    
 [3] "baselineActionReport.bes"                                                                                 
 [4] "bfClientSetting"                                                                                          
 [5] "completedBaseline.bes"                                                                                    
 [6] "configNquery"                                                                                             
 [7] "configNquery.ipynb"                                                                                       
 [8] "configNquery.py"                                                                                          
 [9] "createComputerGroup.py"                                                                                   
[10] "createComputerGroup2.py"                                                                                  
[11] "credentials.json"                                                                                         
[12] "cycleGather.py"                                                                                           
[13] "cycleGather.xml"                                                                                          
[14] "doc"                                                                                                      
[15] "executeActionList4IEM.py"                                                                                 
[16] "file.xml"                                                                                                 
[17] "fixedFixlets.txt"                                                                                         
[18] "getAssetsNCheckinTime.py"                                                                                 
[19] "getFixlets2DictUrllib3.py"                                                                                
[20] "getFixletsRest.py"                                                                                        
[21] "getFixletsRest2File.py"                                                                                   
[22] "getFixletsRest2File2.py"                                                                                  
[23] "getFixletsRestAdonix.py"                                                                                  
[24] "getFixletsRestAdonixUrllib.py"                                                                            
[25] "getFixletsRestGrass.py"                                                                                   
[26] "Grasskeet-CriticalVulns-baseline.bes"                                                                     
[27] "importTweedleDeeFixlet.py"                                                                                
[28] "importTweedleDeeFixlet3.py"                                                                               
[29] "invokeBesProvisioning.xml"                                                                                
[30] "InvokeFixletRestViaPython.docx"                                                                           
[31] "InvokeFixletRestViaPython.pdf"                                                                            
[32] "InvokeFixletRestViaPython2.pdf"                                                                           
[33] "Li3000.txt"                                                                                               
[34] "MasterSite_FixletA.bes"                                                                                   
[35] "MasterSite_FixletB.bes"                                                                                   
[36] "MS11-092_ Vulnerability in Windows Media

In [52]:
library("reticulate")
source_python("configNqueryCmdline.py")

In [53]:
# Set up and read the credentials-file, returning the configuration-dictionary
rCfg<-"../credentials.json"
myCfg<-readConfig(rCfg)

# Create a 'relevance string and query the BigFix server
newRelevance<-"names whose(it as lowercase contains %22adhay%22) of bes computers"
xmlResult<-queryViaRelevance(myCfg,newRelevance)

# Using the the lambda function we defined for parsing bes computers, print out the first 6 elements of the DataFrame
print(head(computersLf1(xmlResult)))

             0
1   ADHAYGAP21
2   ADHAYVCR01
3   ADHAYFDB01
4 ADHAYMCS01N1
5   ADHAYGAP16
6   ADHAYWEB08
